In [ ]:
# File: five-twist.ipynb
# Code: Claude Code and Codex
# Review: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
import numpy as np
from frontend import App

# create an app
app = App.create("five-twist")

# create a cylinder mesh with specified radius and length
half_length = 2.6
V, F = app.mesh.cylinder(r=0.55, min_x=-half_length, max_x=half_length, n=180)
app.asset.add.tri("cylinder", V, F)

# create a scene
scene = app.scene.create()
objs = []

# arrange 5 cylinders in a circular pattern
n, r = 5, 1
for i in range(5):
    t = 2 * i / n * np.pi
    x, y = r * np.cos(t), r * np.sin(t)
    objs.append(scene.add("cylinder").at(0, x, y).jitter())

# configure material properties and animations for each cylinder
for obj in objs:

    # set bending stiffness, contact gap, Young's modulus, Poisson's ratio and density
    (
        obj.param.set("bend", 500.0)
        .set("contact-gap", 4e-3)
        .set("young-mod", 1e4)
        .set("poiss-rat", 0.25)
        .set("density", 3.5)
    )

    # define animation parameters for twisting motion
    angular_vel, move_delta, scale, t_end, t_wait = 360, 1, 0.75, 25, 5
    x, y, z = obj.position

    # pin and animate left end - spin, scale and move
    left_pin = obj.pin(obj.grab([-1, 0, 0]))
    left_pin.spin(axis=[1, 0, 0], angular_velocity=angular_vel, t_end=t_wait).move_by(
        [move_delta, 0, 0], 0.0, t_end
    )
    left_pin.scale(scale, 0.0, t_wait, center=[-x-half_length, -y, -z])

    # pin and animate right end - spin in opposite direction, scale and move
    right_pin = obj.pin(obj.grab([1, 0, 0]))
    right_pin.spin(axis=[-1, 0, 0], angular_velocity=angular_vel, t_end=t_wait).move_by(
        [-move_delta, 0, 0], 0.0, t_end
    )
    right_pin.scale(scale, 0.0, t_wait, center=[-x+half_length, -y, -z])

    # add additional spinning motion
    angular_vel, t_end = 180, 15

    left_pin.spin(
        center=[-x, -y, -z],
        axis=[1, 0, 0],
        angular_velocity=angular_vel,
        t_start=t_wait,
        t_end=t_end,
    )

    right_pin.spin(
        center=[-x, -y, -z],
        axis=[-1, 0, 0],
        angular_velocity=angular_vel,
        t_start=t_wait,
        t_end=t_end,
    )

# compile the scene and report stats
scene = scene.build().report()

# preview the initial scene
scene.preview()

In [ ]:
# create a new session with the compiled scene
session = app.session.create(scene)

# set session parameters - timestep, frames, disable gravity
(
    session.param.set("dt", 1 / 60)
    .set("frames", 480)
    .set("gravity", 0.0)
)

# build this session
session = session.build()

In [ ]:
# start the simulation and live-preview the results
session.start().preview()

# also show simulation logs in realtime
session.stream()

In [ ]:
# create an animation from the simulation results
session.animate()

In [ ]:
# export the animation to file
session.export.animation()

In [ ]:
# this is for CI
if app.ci:
    assert session.finished()